<a href="https://colab.research.google.com/github/batiukmaks/Machine-Learning-Labs/blob/main/labs/lab-1/ML_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab: https://docs.google.com/document/d/1M7k-qFUd6GukcvrLyY0P0iwoZlE6q_2MAPJaQ5xuwMQ/edit

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd

In [19]:
# parch	# of parents / children aboard the Titanic	- how many children & parents of the passenger aboard the Titanic.
# fare	Passenger fare
titanic_df = sns.load_dataset("titanic")

In [89]:
# Select columns given in the task
selected_columns = ['parch','age', 'fare']
selected_df = titanic_df[selected_columns]

# Filter out columns with NaNs
cleaned_df = selected_df.dropna()

In [65]:
class Zscore:
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def get_scores(self, col):
        column = self.data[col]
        scores = (column - column.mean()) / column.std()
        return scores

    def get_average_scores(self, cols):
        scores = np.array([self.get_scores(col) for col in cols])
        average_scores = np.mean(scores, axis=0)
        return average_scores

In [81]:
# Calculate Z-score for the rows
score_calc = Zscore(cleaned_df)
avg_scores = score_calc.get_average_scores(selected_columns)
cleaned_df["z_score"] = avg_scores

<ipython-input-81-7c8401ec73c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["z_score"] = avg_scores


In [85]:
ALTERNATIVE_EXTREME_ROWS_NUM = 7

# Retrieve ourliers
outliers_df = cleaned_df[cleaned_df.z_score > 3]
print(f"There is [{outliers_df.shape[0]}] outliers with z-score > 3")

# Retrieve top rows as outliers if there is less than 5 outliers retrieved
if outliers_df.shape[0] < 5:
    indices = avg_scores.argsort()[-ALTERNATIVE_EXTREME_ROWS_NUM:]
    outliers_df = cleaned_df.iloc[indices]
    print(f"Selected [{ALTERNATIVE_EXTREME_ROWS_NUM}] rows with top Z-scores as outliers.")

# DF without outliers
normal_df = cleaned_df.drop(outliers_df.index)
outliers_df

There is [7] outliers with z-score > 3
Selected [7] rows with top Z-scores as outliers.


,parch,age,fare,z_score
299,1,50.0,247.5208,2.028548
638,5,41.0,39.6875,2.075479
678,6,43.0,46.9000,2.557449
737,0,35.0,512.3292,2.961717
258,0,35.0,512.3292,2.961717
679,1,36.0,512.3292,3.375309
438,4,64.0,263.0000,3.619238


In [87]:
import plotly.graph_objs as go

# Create a trace for outliers
trace1 = go.Scatter3d(
    x = outliers_df['parch'],
    y = outliers_df['age'],
    z = outliers_df['fare'],
    mode = 'markers',
    marker = dict(size=4, color='red')
)

# Create a trace for the rest of the data
trace2 = go.Scatter3d(
    x = normal_df['parch'],
    y = normal_df['age'],
    z = normal_df['fare'],
    mode = 'markers',
    marker = dict(size=2, color='blue')
)

data = [trace1, trace2]

layout = go.Layout(
    title='Scatter Plot of Z-Scores',
    scene=dict(
        xaxis=dict(title='parch'),
        yaxis=dict(title='age'),
        zaxis=dict(title='fare')
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()